In [10]:
import pandas as pd
import numpy as np
import matplotlib as mpb
import seaborn as sns
from scipy.interpolate import interp1d, CubicSpline
import xlwings as xw

In [11]:
app = xw.App()
filepath = r"C:\Users\nnamd\OneDrive\Documents\AvE Consulting\Technical\NAS\Yield Curve\August 2023\2023.08.3 RISK FREE YC JUN 2022 - AUG 2023_3108.xlsm"



In [12]:
wb = xw.Book(filepath)

In [13]:
list(wb.sheets)

[<Sheet [2023.08.3 RISK FREE YC JUN 2022 - AUG 2023_3108.xlsm]SW - Tables>,
 <Sheet [2023.08.3 RISK FREE YC JUN 2022 - AUG 2023_3108.xlsm]SW - Tables (Monthly CSBC)>,
 <Sheet [2023.08.3 RISK FREE YC JUN 2022 - AUG 2023_3108.xlsm]SW - Tables (Monthly Linear)>,
 <Sheet [2023.08.3 RISK FREE YC JUN 2022 - AUG 2023_3108.xlsm]LinearInterpolation>,
 <Sheet [2023.08.3 RISK FREE YC JUN 2022 - AUG 2023_3108.xlsm]Sheet1>,
 <Sheet [2023.08.3 RISK FREE YC JUN 2022 - AUG 2023_3108.xlsm]CubicSplineInterpolation_CSBC>,
 <Sheet [2023.08.3 RISK FREE YC JUN 2022 - AUG 2023_3108.xlsm]FMDQ - Interpolated>,
 <Sheet [2023.08.3 RISK FREE YC JUN 2022 - AUG 2023_3108.xlsm]Graphs>]

In [5]:
#Not using this pandas command for now as using xlwings method
#df = pd.read_excel(r'C:\Users\nnamd\OneDrive\Documents\AvE Consulting\Technical\NAS\Yield Curve\August 2023\2023.08.3 RISK FREE YC JUN 2022 - AUG 2023_3108.xlsm', header = 5, sheet_name = "SW - Tables", usecols="DQ:BHB", index_col = 0, nrows = 15, parse_dates = False)

In [14]:
sheet = wb.sheets['SW - Tables'] 
df = sheet['ForwardCurveRaw'].expand().options(pd.DataFrame).value
df


,0.000000,0.083333,0.166667,0.250000,0.333333,0.416667,0.500000,0.583333,0.666667,0.750000,...,119.250000,119.333333,119.416667,119.500000,119.583333,119.666667,119.750000,119.833333,119.916667,120.000000
Tenor,,,,,,,,,,,,,,,,,,,,,
2022-06-30,0.062923,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0.131505
2022-07-31,0.069604,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0.134900
2022-08-31,0.069093,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0.136461
2022-09-30,0.091597,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0.143402
2022-10-31,0.151480,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0.149493
2022-11-30,0.159586,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0.147235
2022-12-31,0.084614,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0.140003
2023-01-31,0.039068,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0.149582
2023-02-28,0.038323,None,None,None,None,None,None,None,None,None,...,None,None,None,None,None,None,None,None,None,0.152000


In [15]:
#
#df.index = pd.to_datetime(df.index)
df = df.apply(pd.to_numeric)

In [8]:
print(df.dtypes)

0.000000      float64
0.083333      float64
0.166667      float64
0.250000      float64
0.333333      float64
               ...   
119.666667    float64
119.750000    float64
119.833333    float64
119.916667    float64
120.000000    float64
Length: 1441, dtype: object


In [9]:
 print(df.index)

DatetimeIndex(['2022-06-30', '2022-07-31', '2022-08-31', '2022-09-30',
               '2022-10-31', '2022-11-30', '2022-12-31', '2023-01-31',
               '2023-02-28', '2023-03-31', '2023-04-30', '2023-05-31',
               '2023-06-30', '2023-07-31', '2023-08-31'],
              dtype='datetime64[ns]', name='Tenor', freq=None)


In [10]:
%whos

Variable      Type         Data/Info
------------------------------------
CubicSpline   type         <class 'scipy.interpolate._cubic.CubicSpline'>
app           App          <App [excel] 21808>
df            DataFrame                0.000000    0<...>n[15 rows x 1441 columns]
filepath      str          C:\Users\nnamd\OneDrive\D<...>2022 - AUG 2023_3108.xlsm
interp1d      type         <class 'scipy.interpolate._interpolate.interp1d'>
mpb           module       <module 'matplotlib' from<...>matplotlib\\__init__.py'>
np            module       <module 'numpy' from 'C:\<...>ges\\numpy\\__init__.py'>
pd            module       <module 'pandas' from 'C:<...>es\\pandas\\__init__.py'>
sheet         Sheet        <Sheet [2023.08.3 RISK FR<...>23_3108.xlsm]SW - Tables>
sns           module       <module 'seaborn' from 'C<...>s\\seaborn\\__init__.py'>
wb            Book         <Book [2023.08.3 RISK FRE<...>22 - AUG 2023_3108.xlsm]>
xw            module       <module 'xlwings' from 'C<...>s\\xlwi

In [11]:
# # this allows you to load a highlighted range in Excel into Python. So i just highlight the forward rates in the workbook and this will load them in
# df = xw.load()
# df = df.apply(pd.to_numeric) #the purpose of this is to convert the loaded array into a numeric dataframe that can be interpolated

In [12]:
# ''''''@xw.func
# @xw.arg(X_train, np.array, ndim = 2)
# def Cubic_Splines(X_train):
#     ''''''

In [13]:
df

,0.000000,0.083333,0.166667,0.250000,0.333333,0.416667,0.500000,0.583333,0.666667,0.750000,...,119.250000,119.333333,119.416667,119.500000,119.583333,119.666667,119.750000,119.833333,119.916667,120.000000
Tenor,,,,,,,,,,,,,,,,,,,,,
2022-06-30,0.062923,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.131505
2022-07-31,0.069604,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.134900
2022-08-31,0.069093,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.136461
2022-09-30,0.091597,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.143402
2022-10-31,0.151480,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.149493
2022-11-30,0.159586,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.147235
2022-12-31,0.084614,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.140003
2023-01-31,0.039068,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.149582
2023-02-28,0.038323,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.152000


In [14]:
df1 = df.interpolate(method = 'linear', axis=1)

In [15]:
df1

,0.000000,0.083333,0.166667,0.250000,0.333333,0.416667,0.500000,0.583333,0.666667,0.750000,...,119.250000,119.333333,119.416667,119.500000,119.583333,119.666667,119.750000,119.833333,119.916667,120.000000
Tenor,,,,,,,,,,,,,,,,,,,,,
2022-06-30,0.062923,0.062923,0.062923,0.062923,0.062923,0.062923,0.062923,0.062923,0.062923,0.062923,...,0.131505,0.131505,0.131505,0.131505,0.131505,0.131505,0.131505,0.131505,0.131505,0.131505
2022-07-31,0.069604,0.069604,0.069604,0.069604,0.069604,0.069604,0.069604,0.069604,0.069604,0.069604,...,0.134900,0.134900,0.134900,0.134900,0.134900,0.134900,0.134900,0.134900,0.134900,0.134900
2022-08-31,0.069093,0.069093,0.069093,0.069093,0.069093,0.069093,0.069093,0.069093,0.069093,0.069093,...,0.136461,0.136461,0.136461,0.136461,0.136461,0.136461,0.136461,0.136461,0.136461,0.136461
2022-09-30,0.091597,0.091597,0.091597,0.091597,0.091597,0.091597,0.091597,0.091597,0.091597,0.091597,...,0.143402,0.143402,0.143402,0.143402,0.143402,0.143402,0.143402,0.143402,0.143402,0.143402
2022-10-31,0.151480,0.151480,0.151480,0.151480,0.151480,0.151480,0.151480,0.151480,0.151480,0.151480,...,0.149493,0.149493,0.149493,0.149493,0.149493,0.149493,0.149493,0.149493,0.149493,0.149493
2022-11-30,0.159586,0.159586,0.159586,0.159586,0.159586,0.159586,0.159586,0.159586,0.159586,0.159586,...,0.147235,0.147235,0.147235,0.147235,0.147235,0.147235,0.147235,0.147235,0.147235,0.147235
2022-12-31,0.084614,0.084614,0.084614,0.084614,0.084614,0.084614,0.084614,0.084614,0.084614,0.084614,...,0.140003,0.140003,0.140003,0.140003,0.140003,0.140003,0.140003,0.140003,0.140003,0.140003
2023-01-31,0.039068,0.039068,0.039068,0.039068,0.039068,0.039068,0.039068,0.039068,0.039068,0.039068,...,0.149582,0.149582,0.149582,0.149582,0.149582,0.149582,0.149582,0.149582,0.149582,0.149582
2023-02-28,0.038323,0.038323,0.038323,0.038323,0.038323,0.038323,0.038323,0.038323,0.038323,0.038323,...,0.152000,0.152000,0.152000,0.152000,0.152000,0.152000,0.152000,0.152000,0.152000,0.152000


In [16]:
# This line is giving an error
df2 = df.interpolate(method ='cubic', axis = 1)

ValueError: Index column must be numeric or datetime type when using cubic method other than linear. Try setting a numeric or datetime index column before interpolating.

In [ ]:
#df2

In [17]:
df_Arr = df.dropna(axis=1).to_numpy()
x = np.asarray(df.dropna(axis = 1).columns)

In [ ]:
f3a = CubicSpline(x,df_Arr, axis = 1)
f3b = CubicSpline(x,df_Arr, axis = 1, bc_type='natural')

In [ ]:
x_new  = np.linspace(0, 120, 1441)
y_new3a, y_new3b = f3a(x_new), f3b(x_new)
df3a, df3b = pd.DataFrame(data = y_new3a, index = df.index, columns = x_new), pd.DataFrame(data = y_new3b, index = df.index, columns = x_new)

In [ ]:
# This line is giving an error
# df4 = df.interpolate(method ='quadratic', axis = 1)

In [ ]:
wb.sheets['LinearInterpolation'].range('A1').value = df1
wb.sheets['CubicSplineInterpolation_CSBC'].range('A1').value = df3b

In [ ]:
# #I had an issue with this code whereby Ithe excel file was held up and couldn't open or close. I will try to use a book instance using xlwings to access the file

# with pd.ExcelWriter(r'C:\Users\nnamd\OneDrive\Documents\AvE Consulting\Technical\NAS\Yield Curve\August 2023\2023.08.3 RISK FREE YC.xlsm', mode='a', if_sheet_exists="overlay") as writer:
#     df1.to_excel(writer, sheet_name='LinearInterpolation')
#     df3b.to_excel(writer, sheet_name='CubicSplineInterpolation_CSBC')
    

In [ ]:
# with pd.ExcelWriter(r'C:\Users\nnamd\OneDrive\Documents\AvE Consulting\Technical\NAS\Yield Curve\August 2023\InterpolatedYC.xlsx',
#                     mode='w', if_sheet_exists="overlay") as writer:

In [ ]:
#This allows me to see all the variables in the session
%whos